In [1]:
#Importing packages and checking GPU
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset,random_split,ConcatDataset
from torchsummary import summary
import numpy as np
from PIL import Image,ImageDraw,ImageFont
import os
from sklearn.model_selection import train_test_split

if torch.cuda.is_available():
	print("PyTorch is using the GPU")
	GPUCount = torch.cuda.device_count()
	print(f"Found {GPUCount} GPUs")

	for i in range(GPUCount):
		print(f"GPU {i} found: {torch.cuda.get_device_name(i)}")

	device = torch.device("cuda:0")
else:
	print("PyTorch is using the CPU")
	device = torch.device("cpu")

print(f"Selected Device: {device}")

PyTorch is using the GPU
Found 1 GPUs
GPU 0 found: NVIDIA GeForce RTX 5070 Laptop GPU
Selected Device: cuda:0


In [2]:
#Load BuiltIn dataset

Transform = transforms.Compose(
	[
		transforms.Grayscale(num_output_channels=1),
		transforms.ToTensor(),
  		transforms.Normalize((0.5,),(0.5,)),
		transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0),
		transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1))
	])

TrainSet = torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=Transform)
TestSet = torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=Transform)

print(f"Size of EMNIST train data: {len(TrainSet)}")
print(f"Size of EMNIST test data: {len(TestSet)}")

Size of EMNIST train data: 60000
Size of EMNIST test data: 10000


In [3]:
#Load Custom dataset

FontPath = 'fonts'
CustomImages = []
CustomLabels = []

ImagesDirectory = 'CustomDataset'

CustomDataset = ImageFolder(root=ImagesDirectory , transform=Transform)

print(f"Size of custom data: {len(CustomDataset)}")

Size of custom data: 45140


In [4]:
#Combine datasets and split into new ones

Dataset = ConcatDataset([TrainSet,TestSet,CustomDataset])

print(f"Size of custom data: {len(Dataset)}")

TrainSize = int(0.7*(len(Dataset)))
ValSize = int(0.2*(len(Dataset)))
TestSize = len(Dataset)-(TrainSize+ValSize)

print(f"Size of train data: {TrainSize}")
print(f"Size of validation data: {ValSize}")
print(f"Size of test data: {TestSize}")

TrainDataset, ValDataset, TestDataset = random_split(Dataset , [TrainSize, ValSize, TestSize])

TrainLoader = DataLoader(TrainDataset,batch_size=128,shuffle=True)
ValLoader = DataLoader(ValDataset,batch_size=128,shuffle=False)
TestLoader = DataLoader(TestDataset,batch_size=128,shuffle=False)

print("Dataset loaded successfully")

Size of custom data: 115140
Size of train data: 80598
Size of validation data: 23028
Size of test data: 11514
Dataset loaded successfully


In [5]:
#Creating Model

#Define architecture
class MNISTModel(nn.Module):
	def __init__(self):
		super(MNISTModel,self).__init__()

		#Layer1
		self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1,bias=False)#Size does not change
		self.bn1 = nn.BatchNorm2d(num_features=32)
		self.relu1 = nn.ReLU()
		self.pool1 = nn.MaxPool2d(kernel_size=2)#Size halves into 14x14

		#Layer2
		self.conv2 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1,bias=False)
		self.bn2 = nn.BatchNorm2d(num_features=32)
		self.relu2 = nn.ReLU()

		#Layer3
		self.conv3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn3 = nn.BatchNorm2d(num_features=64)
		self.relu3 = nn.ReLU()

		#Layer4
		self.conv4 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn4 = nn.BatchNorm2d(num_features=64)
		self.relu4 = nn.ReLU()

		#Layer5
		self.conv5 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn5 = nn.BatchNorm2d(num_features=64)
		self.relu5 = nn.ReLU()
		self.pool2 = nn.MaxPool2d(kernel_size=2)#Size halves into 7x7

		#FlattenLayer
		self.flatten = nn.Flatten()

		#Layer6
		self.fc1 = nn.Linear(in_features=64*7*7,out_features=64)
		self.relu6 = nn.ReLU()

		#Layer7
		self.fc2 = nn.Linear(in_features=64,out_features=32)
		self.relu7 = nn.ReLU()

		#DropoutLayer
		self.dropout = nn.Dropout(p=0.2)

		#Layer8
		self.fc3 = nn.Linear(in_features=32,out_features=10)

	def forward(self,x):

		#Pass through Layer1
		x = self.pool1(self.relu1(self.bn1(self.conv1(x))))

		#Pass through Layer2
		x = self.relu2(self.bn2(self.conv2(x)))

		#Pass through Layer3
		x = self.relu3(self.bn3(self.conv3(x)))

		#Pass through Layer4
		x = self.relu4(self.bn4(self.conv4(x)))

		#Pass through Layer5
		x = self.pool2(self.relu5(self.bn5(self.conv5(x))))

		#Pass through Layer5
		x = self.flatten(x)

		#Pass through Layer6
		x = self.relu6(self.fc1(x))

		#Pass through Layer7
		x = self.relu7(self.fc2(x))

		#Pass through DropoutLayer
		x = self.dropout(x)

		#Pass through Layer7
		x = self.fc3(x)

		#Return Prediction
		return x

#Create and print summary	
print("Creating MNIST Model...")
model = MNISTModel().to(device)
print("Model Created")

print("Model Summary: ")
summary(model,input_size=(1,28,28))

#Compile model
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Creating MNIST Model...
Model Created
Model Summary: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             288
       BatchNorm2d-2           [-1, 32, 28, 28]              64
              ReLU-3           [-1, 32, 28, 28]               0
         MaxPool2d-4           [-1, 32, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           9,216
       BatchNorm2d-6           [-1, 32, 14, 14]              64
              ReLU-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 64, 14, 14]          18,432
       BatchNorm2d-9           [-1, 64, 14, 14]             128
             ReLU-10           [-1, 64, 14, 14]               0
           Conv2d-11           [-1, 64, 14, 14]          36,864
      BatchNorm2d-12           [-1, 64, 14, 14]             128
             ReLU-13           [-1, 64, 14, 14]  

In [6]:
#Training and saving the model

print("Training Model...")
epochs = 100
patience = 5
counter = 0
MinValidation = float('inf')

for epoch in range(epochs):
	#Train Loop

	#Set model to training mode
	model.train()
	TrainLoss = 0.0

	for i,data in enumerate(TrainLoader,0):
		inputs,labels = data[0].to(device),data[1].to(device)

		#Zero the parameter gradients
		optimizer.zero_grad()

		#Forward pass
		outputs = model(inputs)

		#Calculate loss
		loss = loss_function(outputs,labels)

		#Backward pass
		loss.backward()

		#Update weights
		optimizer.step()

		#Update loss
		TrainLoss += loss.item()
	
	#Validation Loop
	model.eval()
	ValidationLoss = 0.0
	correct = 0
	total = 0

	with torch.no_grad():
		for data in ValLoader:
			images,labels = data[0].to(device),data[1].to(device)
			outputs = model(images)
			loss = loss_function(outputs,labels)
			ValidationLoss += loss.item()

			_,predicted = torch.max(outputs.data,1)
			total += labels.size(0)
			correct += (predicted == labels).sum().item()

		acc = 100*correct/total
		print(f"Epoch: {epoch+1}/{epochs} | "
			f"Training Loss: {TrainLoss/len(TrainLoader):.3f} | "
			f"Validation Loss: {ValidationLoss / len(ValLoader):.3f} | "
			f"Accuracy: {acc:.3f}%")
		
		if ValidationLoss < MinValidation:
			MinValidation = ValidationLoss
			counter = 0
			torch.save(model.state_dict(),"MNISTModel.pth")
			print(f"Validation Loss has improved at epoch {epoch+1}. Model Saved!")
		else:
			counter += 1
			print(f"Validation Loss has not improved. Patience:{counter}/{patience}")
		
		if counter >= patience:
			print("Early stopping triggered!")
			break
		
print("Training Complete")

Training Model...
Epoch: 1/100 | Training Loss: 0.511 | Validation Loss: 0.273 | Accuracy: 91.063%
Validation Loss has improved at epoch 1. Model Saved!
Epoch: 2/100 | Training Loss: 0.245 | Validation Loss: 0.281 | Accuracy: 90.850%
Validation Loss has not improved. Patience:1/5
Epoch: 3/100 | Training Loss: 0.208 | Validation Loss: 0.188 | Accuracy: 93.999%
Validation Loss has improved at epoch 3. Model Saved!
Epoch: 4/100 | Training Loss: 0.192 | Validation Loss: 0.173 | Accuracy: 94.554%
Validation Loss has improved at epoch 4. Model Saved!
Epoch: 5/100 | Training Loss: 0.175 | Validation Loss: 0.165 | Accuracy: 94.793%
Validation Loss has improved at epoch 5. Model Saved!
Epoch: 6/100 | Training Loss: 0.165 | Validation Loss: 0.152 | Accuracy: 95.123%
Validation Loss has improved at epoch 6. Model Saved!
Epoch: 7/100 | Training Loss: 0.160 | Validation Loss: 0.139 | Accuracy: 95.388%
Validation Loss has improved at epoch 7. Model Saved!
Epoch: 8/100 | Training Loss: 0.154 | Valida

In [10]:
#Testing Model

print("Testing model with inbuilt dataset...")

model.load_state_dict(torch.load("MNISTModel.pth"))

TestLoss = 0.0
correct = 0
total = 0

with torch.no_grad():
	for images, labels in TestLoader:

		#Move data to device
		images,labels = images.to(device),labels.to(device)

		#Forward pass
		outputs = model(images)

		#Calculate loss
		loss = loss_function(outputs,labels)
		TestLoss += loss.item()

		#Get predicted class
		_,predicted = torch.max(outputs.data,1)

		#Update total and correct counts
		total += labels.size(0)
		correct += (predicted == labels).sum().item()
	
#Test Results

FinalLoss = TestLoss/len(TestLoader)
FinalAcc = 100*correct/total

print(f"Test Accuracy: {FinalAcc:.3f}%")
print(f"Test Loss: {FinalLoss:.3f}")

Testing model with inbuilt dataset...
Test Accuracy: 96.309%
Test Loss: 0.114
